In [1]:
import numpy as np
import pandas as pd

In [2]:
#order of calculation of f based on arcs, explain them
arc_f={(3,2):[(1,3)], (3,4):[(1,3)],(5,2): [(1,5)], (5,6): [(1,5)],(2,4):[(3,2),(1,2),(5,2)],(2,7):[(3,2),(1,2),(5,2)],(2,6):[(3,2),(1,2),(5,2)], (4,7):[(2,4),(3,4)], (6,7):[(2,6),(5,6)]} # ordered arcs
#tij
arc_tij={(1,2): 10, (1,3):20, (1,5):25, (2,4):35, (2,6):35, (2,7):30, (3,2):15, (3,4):20, (4,7):30, (5,2):20, (5,6):25,  (6,7):10 }
#alpha, beta
node_albe={1:(0,1000000), 2:(18,30), 3:(35,45), 4:(64,75), 5:(35,50), 6:(64,75), 7:(0,1000000)}
#gamma, delta
arc_gamdel={(1,2): (10,15),(1,3):(10,25), (1,5):(15,25), (2,4):(28,35), (2,6):(26,34), (2,7):(32,60), (3,2):(35,50), (3,4):(43,50), (4,7):(70,99), (5,2): (30,80), (5,6):(48,55), (6,7):(72,80)}

In [3]:
#feasibility function
def feasib(fij,alph_i,beta_i,gam_ij,delta_ij):#conditions from (1) to (5)
    if ((alph_i <= gam_ij) and (alph_i <= fij) and (fij<= beta_i)):#condition (1), not using (alph_i <= gam_ij), for (5,2), for (2,7) would not be correct
        l_pi= max(0,gam_ij-fij)#lower pi
        u_pi=min(beta_i-fij,delta_ij-fij)#upper pi
        if l_pi <=u_pi:
            return True
        else:
            return False
    #condition (2)and(4)
    elif ((beta_i<=fij) and (gam_ij<=fij) and (fij <= delta_ij)) or ((gam_ij<fij) and (fij<alph_i)and(alph_i<=delta_ij)) or ((gam_ij<fij)and (fij<delta_ij)and(delta_ij<=alph_i)):
        return True
    #conditions(3) and (5)
    elif ((fij<gam_ij)and (gam_ij<alph_i))or((fij<alph_i)and(alph_i<gam_ij)and(gam_ij<beta_i))or((fij<alph_i)and(alph_i<beta_i)and(beta_i<gam_ij))or(delta_ij<fij):
        return False

In [4]:
#check feasibility
def check_feasibility(f_ij, tc_ij, a_i,a_j,b_i,b_j,g_ij,d_ij,g_jk,d_jk): # tc=travelcost, a=alpha, b=beta, g=gamma, d=delta
    p_i=None
    b=False
    f=None
    #if ((alph_i <= gam_ij) and (alph_i <= fij) and (fij<= beta_i)):#condition (1), not using (alph_i <= gam_ij)
    #1st condition
    if ((a_i <= f_ij) and (f_ij<= b_i)):
        lower_p_i= max(0,g_ij-f_ij)
        upper_p_i=min(b_i-f_ij,d_ij-f_ij)
        if lower_p_i>upper_p_i:
            return p_i, b, f
        else:
            for p_i in range (lower_p_i,upper_p_i+1):
                f_jk= f_ij+tc_ij+p_i
                # insert if statement: if f_jk is feasible (=satisfy conditions (1), (2), (4)) 
    elif ((b_i<=f_ij) and (f_ij <= d_ij) and (g_ij<=f_ij)) or ((g_ij<f_ij) and (f_ij<a_i)and(a_i<=d_ij)) or ((g_ij<f_ij)and (f_ij<d_ij)and(d_ij<=a_i)): #2nd and 4th condition
        p_i=0
        b=True
        f_jk= f_ij+tc_ij+p_i
        return f_jk,p_i,b
    elif ((f_ij<g_ij)and (g_ij<a_i))or((f_ij<a_i)and(a_i<g_ij)and(g_ij<b_i))or((f_ij<a_i)and(a_i<b_i)and(b_i<g_ij))or(d_ij<f_ij): #3rd and 5th condition
        k=False
        return pi, b , f

In [5]:
#arc_f={(3,2):[(1,3)], (3,4):[(1,3)],(5,2): [(1,5)], (5,6): [(1,5)],(2,4):[(3,2),(1,2),(5,2)],(2,7):[(3,2),(1,2),(5,2)],(2,6):[(3,2),(1,2),(5,2)], (4,7):[(2,4),(3,4)], (6,7):[(2,6),(5,6)]} # ordered arcs
#dict of values of f
dict_f={(1,3):(0,10),(1,2):(0,10), (1,5):(0,15)}#store the value of f(i,j) and add up the current f(j,k)
k=0
N=9 #number of iterations for the for loop #we have 12-3=9 arcs to calculate
#feasib(fij,alph_i,beta_i,gam_ij,delta_ij)
for key in arc_f:
    k+=1
    #print(key[0],key[1])#j=key[0], k=key[1]
    #print(key)
    prev_arc=arc_f[key]
    f_list=[]
    fpi_list=[]
    for arc in prev_arc:
        #print(arc)
        fij=dict_f[arc][0]
        #print('fij=',fij)
        tij=arc_tij[arc]
        #print('tij=',tij)
        alph_i, beta_i= node_albe[arc[0]][0],node_albe[arc[0]][1]
        #print('alph_i, beta_i=',node_albe[arc[0]][0],node_albe[arc[0]][1])
        
        #alph_j, beta_j
        alph_j, beta_j= node_albe[key[0]][0],node_albe[key[0]][1]
        #print('alph_i, beta_i=',node_albe[arc[0]][0],node_albe[arc[0]][1])
        
        
        gam_ij,delta_ij=arc_gamdel[arc][0], arc_gamdel[arc][1]
        #print('gam_ij,delta_ij=',arc_gamdel[arc][0],  arc_gamdel[arc][1])
        
        #gam_jk,delta_jk
        gam_jk,delta_jk=arc_gamdel[key][0], arc_gamdel[key][1]
        
        #print(feasib(fij,alph_i,beta_i,gam_ij,delta_ij)) 
        
        #cal_feasib_fjk
        print(cal_feasib_fjk(fij,alph_i,alph_j,beta_i,beta_j,gam_ij,delta_ij,gam_jk,delta_jk,tij))
        f, pi, b= cal_feasib_fjk(fij,alph_i,alph_j,beta_i,beta_j,gam_ij,delta_ij,gam_jk,delta_jk,tij)
        #f_list.append()
        #append to dict of f_list
        if b==True:
            fpi_list.append((f, pi))
            f_list.append(f)
    #look for the min f
    index_min = np.argmin(f_list)
    dict_f[key]=fpi_list [index_min]
    if k>=N:
        break
dict_f

(35, 15, True)
(35, 15, True)
(50, 25, True)
(40, 15, True)
(None, 10, False)
(20, 10, True)
(None, 0, False)
(50, 0, True)
(None, 15, False)
(None, 0, False)
(None, 10, False)
(20, 10, True)
(None, 0, False)
(64, 9, True)
(64, 9, True)
(64, 9, True)
(73, 8, True)


{(1, 3): (0, 10),
 (1, 2): (0, 10),
 (1, 5): (0, 15),
 (3, 2): (35, 15),
 (3, 4): (35, 15),
 (5, 2): (50, 25),
 (5, 6): (40, 15),
 (2, 4): (20, 10),
 (2, 7): (50, 0),
 (2, 6): (20, 10),
 (4, 7): (64, 9),
 (6, 7): (64, 9)}

### look at the whole requirements of task 2 again